In [592]:
import numpy as np
import pandas as pd
import random
from IPython.display import display


#### Load the matrices

In [593]:
battery_expenditure = pd.read_csv('battery_expenditure_matrix.csv')
distance_matrix = pd.read_csv('distance_matrix.csv')
time_matrix = pd.read_csv('time_matrix.csv')

In [594]:
customer_demands = [[3,4],[5,15],[3,7],[8,10],[6,13]]
# customer_demands = [[3,4],[5,15],[3,7],[8,7],[6,13]]
# customer_demands = [[3,15],[5,15],[3,15],[8,15],[6,15]]

In [595]:
covered_nodes = []

for customer_demand in customer_demands:
    covered_nodes.append(customer_demand[0])
    covered_nodes.append(customer_demand[1])
    sorted(covered_nodes)


#### Generating random combination

In [596]:
num_sequences=20
start_node=0
end_node=16
battery_threshold=10

w1=0.5
w2=0.5

In [597]:
def carCapacityMaintained(route):
    curWt = 0
    for i in range(len(route)):
        wt = route[i][1]
        curWt += wt
        if curWt > 3:
            return False
        if curWt < 0:
            return False
         
    return True


In [598]:
def allCustomersDemand(sequence):
    route=[]
    for node in sequence[1:-1]:
        route.append(node[0])
    return sorted(route) == sorted(covered_nodes)

In [599]:
def sequenceIsValid(route):
    # check for validity if every customer reaches destination
    if not allCustomersDemand(route):
        return False

    # check for validity if car strength not increase 3
    if not carCapacityMaintained(route):
        return False
    
    #check if battery level not decrease less than 0
    for i in range(len(route) - 1):
        node1 = route[i][0]
        node2 = route[i + 1][0]
                
        if node1 == 1 or node1 == 2:
            battery_level=78

            battery_consumption = battery_expenditure.iloc[node1, node2]
            battery_level -= battery_consumption
            if battery_level < battery_threshold:
                return False
    return True

In [600]:
def shuffle_array_except_first_last(arr):
    first_element = arr[0]
    last_element = arr[-1]
    internal_elements = arr[1:-1]
    random.shuffle(internal_elements)
    shuffled_array = [first_element] + internal_elements + [last_element]
    return shuffled_array

In [601]:
def generate_random_sequences(num_sequences, battery_threshold,customer_demands=customer_demands):
    sequences = []

    for _ in range(num_sequences):
        while True:
            route = [(0, 0) for _ in range(12)]
            used_positions = set()
            unused_positions = set()
            for i in range(12):
                unused_positions.add(i)
            unused_positions.remove(0)
            unused_positions.remove(11)
            used_positions.add(0)
            used_positions.add(11)

            for demand in customer_demands:
                position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                while position1 in used_positions:
                    position1 = random.randint(min(unused_positions),max(unused_positions)-1)
                unused_positions.remove(position1)
                used_positions.add(position1)

                position2 = random.randint(position1 + 1, max(unused_positions))
                while position2 in used_positions:
                    position2 = random.randint(position1 + 1, max(unused_positions))
                unused_positions.remove(position2)
                used_positions.add(position2)

                route[position1] = (demand[0], +1)  # Insert the start node of the customer demand
                route[position2] = (demand[1], -1)  # Insert the end node of the customer demand

            
            if sequenceIsValid(route):
                sequences.append(route)
                break

    return sequences

In [602]:
random_seq=generate_random_sequences(num_sequences,10)
random_seq

[[(0, 0),
  (6, 1),
  (13, -1),
  (8, 1),
  (3, 1),
  (7, -1),
  (10, -1),
  (3, 1),
  (5, 1),
  (4, -1),
  (15, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (8, 1),
  (13, -1),
  (3, 1),
  (7, -1),
  (10, -1),
  (5, 1),
  (3, 1),
  (15, -1),
  (4, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (5, 1),
  (8, 1),
  (13, -1),
  (10, -1),
  (15, -1),
  (3, 1),
  (3, 1),
  (7, -1),
  (4, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (8, 1),
  (5, 1),
  (15, -1),
  (10, -1),
  (3, 1),
  (13, -1),
  (3, 1),
  (4, -1),
  (7, -1),
  (0, 0)],
 [(0, 0),
  (3, 1),
  (5, 1),
  (8, 1),
  (7, -1),
  (15, -1),
  (3, 1),
  (4, -1),
  (6, 1),
  (10, -1),
  (13, -1),
  (0, 0)],
 [(0, 0),
  (6, 1),
  (13, -1),
  (8, 1),
  (3, 1),
  (5, 1),
  (10, -1),
  (15, -1),
  (3, 1),
  (4, -1),
  (7, -1),
  (0, 0)],
 [(0, 0),
  (3, 1),
  (3, 1),
  (8, 1),
  (10, -1),
  (4, -1),
  (7, -1),
  (5, 1),
  (15, -1),
  (6, 1),
  (13, -1),
  (0, 0)],
 [(0, 0),
  (3, 1),
  (6, 1),
  (13, -1),
  (8, 1),
  (3, 1),
  (7, -1),
  (4, -1),
  (5, 1),
  (

In [603]:
def calculate_cost(sequence, time_matrix, distance_matrix, w1, w2):
    cost1 = 0
    cost2 = 0
    cost = 0

    for i in range(len(sequence) - 1):
        node1 = sequence[i][0]
        node2 = sequence[i + 1][0]

        cost1 += time_matrix.iloc[node1, node2]
        cost2 += distance_matrix.iloc[node1, node2]

        cost += (w1 * time_matrix.iloc[node1, node2]) + (w2 * distance_matrix.iloc[node1, node2])

    return pd.DataFrame({'Σt': [cost1], 'Σd': [cost2], 'w1*Σt+w2*Σd': [cost]})

In [604]:
cost_for_each_sequence = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2) for sequence in random_seq]
result_df = pd.concat(cost_for_each_sequence, ignore_index=True)

In [605]:
result_df

,Σt,Σd,w1*Σt+w2*Σd
0,3.319806,153.8683,78.594053
1,3.559278,166.7635,85.161389
2,3.551861,169.7730,86.662431
3,4.139056,204.7431,104.441078
4,3.387361,162.3217,82.854531
5,3.847944,185.5279,94.687922
6,3.483806,165.6748,84.579303
7,3.910472,186.9612,95.435836
8,4.061222,199.5061,101.783661
9,3.521167,170.2619,86.891533


## Genetic Algorithm implementation

In [606]:
def initialize_population(population_size):
    population = generate_random_sequences(population_size,10)
    return population


In [607]:
def tournament_selection(population, fitness_values, tournament_size):
    selected_parents = []
    for _ in range(len(population)):
        tournament_indices = np.random.choice(len(population), tournament_size, replace=False)
        tournament_fitness = [fitness_values[i] for i in tournament_indices]
        winner_index = tournament_indices[np.argmin(tournament_fitness)]
        selected_parents.append(population[winner_index])
    return selected_parents

In [608]:
def cyclic_shift(lst, positions):
    length = len(lst)
    positions = positions % length  # Ensure positions is within the range of list length
    return lst[-positions:] + lst[:-positions]

In [609]:
import random
#Redo
def ordered_crossover(parent1, parent2):
    # Select two random crossover points

    parent1= parent1[1:-1]
    parent2= parent2[1:-1]

    crossover_points = sorted(random.sample(range(1, len(parent1)), 2))
    while True:
        offspring = parent2[crossover_points[0]:crossover_points[1]]
        remaining =[]
        for i in parent1[crossover_points[1]:]:
            if offspring.count(i) + remaining.count(i) < parent1.count(i):
                remaining.append(i)
        
        
        for i in parent1[0:crossover_points[1]]:
            if offspring.count(i) + remaining.count(i) < parent1.count(i):
                remaining.append(i)
        
        offspring = offspring + remaining
        offspring = cyclic_shift(offspring,crossover_points[0])

        offspring= [(0,0)] + offspring + [(0,0)]
        if sequenceIsValid(offspring):
            break
        
        crossover_points = sorted(random.sample(range(1, len(parent1)), 2))

    parent1= [(0,0)] + parent1 + [(0,0)]
    parent2= [(0,0)] + parent2 + [(0,0)]
    return offspring


In [610]:
def inversion_mutation(sequence):
    randPnts = sorted(random.sample(range(1, len(sequence)-2), 2))
    # print("Org: ",sequence)
    while True:
        start = randPnts[0]
        end = randPnts[1]
        
        mutated_sequence = sequence[:start] + list(reversed(sequence[start:end+1])) + sequence[end+1:]
        if sequenceIsValid(mutated_sequence):
            break
        randPnts = sorted(random.sample(range(1, len(sequence)-2), 2))
    # print("Mut: ",mutated_sequence)
    return mutated_sequence

In [611]:
import random

def genetic_algorithm(num_generations, population_size, tournament_size, mutation_probability):
    population = initialize_population(population_size)
    # display(pd.DataFrame(population))
    for generation in range(num_generations):
        fitness_values = [calculate_cost(sequence, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0] for sequence in population]

        parents = tournament_selection(population, fitness_values, tournament_size)
        offspring = []

        for i in range(0, len(parents), 2):
            if i + 1 < len(parents):
                child1 = ordered_crossover(parents[i], parents[i + 1])
                child2 = ordered_crossover(parents[i + 1], parents[i])
                offspring.extend([child1, child2])

        mutated_offspring = []
        for child in offspring:
            if random.uniform(0, 1) < mutation_probability:
                mutated_child = inversion_mutation(child)
                mutated_offspring.append(mutated_child)
            else:
                mutated_offspring.append(child)

        population = mutated_offspring

    best_sequence = min(population, key=lambda x: calculate_cost(x, time_matrix, distance_matrix, w1, w2)['w1*Σt+w2*Σd'].values[0])

    return best_sequence


In [612]:

best_sequence = genetic_algorithm(num_generations=100, population_size=50, tournament_size=5, mutation_probability=0.1)
print("Best Sequence:", best_sequence)
costdf = calculate_cost(best_sequence, time_matrix, distance_matrix, w1, w2)
display(costdf)
print("Best Cost:",costdf.iloc[-1,-1])

Org:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (7, -1), (8, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Mut:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (10, -1), (8, 1), (7, -1), (15, -1), (13, -1), (0, 0)]
Org:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (7, -1), (8, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Mut:  [(0, 0), (6, 1), (3, 1), (10, -1), (8, 1), (7, -1), (5, 1), (4, -1), (3, 1), (15, -1), (13, -1), (0, 0)]
Org:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (7, -1), (8, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Mut:  [(0, 0), (8, 1), (7, -1), (5, 1), (4, -1), (3, 1), (3, 1), (6, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Org:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (7, -1), (8, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Mut:  [(0, 0), (6, 1), (8, 1), (7, -1), (5, 1), (4, -1), (3, 1), (3, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
Org:  [(0, 0), (6, 1), (3, 1), (3, 1), (4, -1), (5, 1), (7, -1), (8, 1), (10, -1), (15, -1), (13, -1), (0, 0)]
M

,Σt,Σd,w1*Σt+w2*Σd
0,2.280889,100.716,51.498444


Best Cost: 51.498444444444445
